In [1]:
from pyspark.sql import SparkSession
from IPython.display import display

# Apache Spark

In [ ]:
spark = SparkSession.builder.appName("Intro_spark").getOrCreate() \
        #.config("spark.driver.bindAddress","localhost") \
        #.config("spark.ui.port","4050") \
        

# check if Adaptive Query Execution (AQE) is enabled
print(spark.conf.get("spark.sql.adaptive.enabled"))

# df = spark.read.parquet("yellow_tripdata_2023-05.parquet")
# print(df.rdd.getNumPartitions())
# df.count()
# df.explain()
# display(df.toPandas())

In [6]:
print(f"{spark.sparkContext.defaultParallelism} cores available")

24 cores available


## Spark Basics

### User Defined Functions (UDFs)

#### Python UDF

Generally the slowest kind of UDF as it applies the function to one row at a time.

### Pandas UDF

Faster than Python UDFs as they apply the function to *batches* of rows (block retrieval) at a time and also because it leverages Apache Arrow for faster serialization (converting the transformed rows back into Java objects to pass back to the JVM).

## Spark Architecture 

In Spark computations are broken into to two rough categories: transformations and actions.

### Transformations

An important thing to note about transformations in Spark is that, generally, they are *lazy* operations; meaning they aren't accualy performed/computed until the resulting dataframe/result set is needed. Generally this means the transformation isn't applied until an *action* is specified. 

There are two types of transformations: narrow and wide transformations. The difference between the two is that *wide* transformations induce what is called a *shuffle* (also called an *exchange*). A shuffle is when a *repartitioning* of the dataset is required. Spark basically works by partitioning the dataset into subsets and having each worker perform the specified computation on a partition of the data. It's easy to think of transformations that would require a shuffle such as the `sort()` operation. Clearly this will require an exchange/shuffle since one needs to compare rows from different partitions to perform a sort. And example of a narrow transformation is the `filter()` transformation. Clearly there is no need to compare rows across partitions to accomplish this operation since we just need to check if a row fulfills the criteria in the filter clause to compute the result set.

Shuffle vs memory partitions

Hierarchy 
    Driver 
        ├── Workers
                ├── Executors
                           ├── Cores

## Structured Streaming

Structured streaming is near real-time streaming since the records are processed in microbatches.

### Streaming aggregations

Streaming aggregation is used to mitigate out-of-memory errors

## Practice questions

These are practice exam questions from the March 2025 practice exam provided by Databricks for the 'Databricks Certified Associate Developer for Apache Spark 3.0 - Python' certification exam. The pdf of the exam is available in the `pdfs/` folder of this section in the JupyterBook.

### Question 1